##### Copyright 2023 The TensorFlow Datasets Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 适用于 Jax 和 PyTorch 的 TFDS

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/datasets/tfless_tfds"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/datasets/tfless_tfds.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/datasets/tfless_tfds.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/datasets/tfless_tfds.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a> </td>
</table>

TFDS 一直都独立于框架。例如，您可以轻松地加载 [NumPy 格式](https://tensorflow.google.cn/datasets/api_docs/python/tfds/as_numpy)的数据集以在 Jax 和 PyTorch 中使用。

TensorFlow 及其数据加载解决方案 ([`tf.data`](https://tensorflow.google.cn/guide/data)) 按照设计是我们 API 中的一等公民。

我们扩展了 TFDS 以支持仅使用 NumPy 而无需 TensorFlow 的数据加载。这对于在 Jax 和 PyTorch 等机器学习框架中使用非常方便。事实上，对于后者的用户来说，TensorFlow：

- 会保留 GPU/TPU 内存；
- 会在 CI/CD 中增加构建时间；
- 在运行时需要花费时间导入。

TensorFlow 不再是读取数据集的依赖项。

机器学习流水线需要一个数据加载器来加载样本，将其解码并呈现给模型。数据加载器使用“源/采样器/加载器”范式：

```
 TFDS dataset       ┌────────────────┐
   on disk          │                │
        ┌──────────►│      Data      │
|..|... │     |     │     source     ├─┐
├──┼────┴─────┤     │                │ │
│12│image12   │     └────────────────┘ │    ┌────────────────┐
├──┼──────────┤                        │    │                │
│13│image13   │                        ├───►│      Data      ├───► ML pipeline
├──┼──────────┤                        │    │     loader     │
│14│image14   │     ┌────────────────┐ │    │                │
├──┼──────────┤     │                │ │    └────────────────┘
|..|...       |     │     Index      ├─┘
                    │    sampler     │
                    │                │
                    └────────────────┘
```

- 数据源负责实时访问和解码来自 TFDS 数据集的样本。
- 索引采样器负责确定记录处理的顺序。在读取任何记录之前，实现全局转换（例如全局重排、分片、重复多个周期）非常重要。
- 数据加载器通过利用数据源和索引采样器来编排加载。它可以实现性能优化（例如，预提取、多进程或多线程）。


## 速览

`tfds.data_source` 是一个用于创建数据源的 API：

1. 用于纯 Python 流水线的快速原型设计；
2. 用于大规模管理数据密集型机器学习流水线。

## 安装

让我们安装并导入所需依赖项：

In [ ]:
!pip install array_record
!pip install tfds-nightly

import os
os.environ.pop('TFDS_DATA_DIR', None)

import tensorflow_datasets as tfds

## 数据源

数据源基本上是 Python 序列。因此，它们需要实现以下协议：

```python
class RandomAccessDataSource(Protocol):
  """Interface for datasources where storage supports efficient random access."""

  def __len__(self) -> int:
    """Number of records in the dataset."""

  def __getitem__(self, record_key: int) -> Sequence[Any]:
    """Retrieves records for the given record_keys."""
```

**警告**：该 API 仍在积极开发中。特别是，`__getitem__` 目前在输入中必须支持 `int` 和 `list[int]`。将来，按照[标准](https://docs.python.org/3/reference/datamodel.html#object.__getitem__)，它可能仅支持 `int`。

底层文件格式需要支持高效的随机访问。目前，TFDS 依赖于 [`array_record`](https://github.com/google/array_record)。

[`array_record`](https://github.com/google/array_record) 是一种衍生自 [Riegeli](https://github.com/google/riegeli) 的新文件格式，实现了 IO 效率的新前沿。特别是，ArrayRecord 支持按记录索引并行读取、写入和随机访问。ArrayRecord 建立在 Riegeli 之上，并支持相同的压缩算法。

[`fashion_mnist`](https://tensorflow.google.cn/datasets/catalog/fashion_mnist) 是一个常见的计算机视觉数据集。要使用 TFDS 检索基于 ArrayRecord 的数据源，只需使用以下命令：

In [ ]:
ds = tfds.data_source('fashion_mnist')

`tfds.data_source` 是一个方便的包装器。它等同于：

In [ ]:
builder = tfds.builder('fashion_mnist', file_format='array_record')
builder.download_and_prepare()
ds = builder.as_data_source()

这将输出一个数据源字典：

```
{
  'train': DataSource(name=fashion_mnist, split='train', decoders=None),
  'test': DataSource(name=fashion_mnist, split='test', decoders=None),
}
```

一旦 `download_and_prepare` 运行并在您生成记录文件后，我们就不再需要 TensorFlow 了。一切都将在 Python/NumPy 中完成！

让我们通过卸载 TensorFlow 并在另一个子进程中重新加载数据源对此进行检查：

In [ ]:
!pip uninstall -y tensorflow

In [ ]:
%%writefile no_tensorflow.py
import os
os.environ.pop('TFDS_DATA_DIR', None)

import tensorflow_datasets as tfds

try:
  import tensorflow as tf
except ImportError:
  print('No TensorFlow found...')

ds = tfds.data_source('fashion_mnist')
print('...but the data source could still be loaded...')
ds['train'][0]
print('...and the records can be decoded.')

In [ ]:
!python no_tensorflow.py

在未来的版本中，我们还将使数据集准备不再依赖 TensorFlow。

数据源的长度为：

In [ ]:
len(ds['train'])

访问数据集的第一个元素：

In [ ]:
%%timeit
ds['train'][0]

…开销与访问任何其他元素一样低。下面是[随机访问](https://en.wikipedia.org/wiki/Random_access)的定义：

In [ ]:
%%timeit
ds['train'][1000]

特征现在使用 NumPy DType（而不是 TensorFlow DType）。您可以使用以下命令检查特征：

In [ ]:
features = tfds.builder('fashion_mnist').info.features

您可以在我们的[文档](https://tensorflow.google.cn/datasets/api_docs/python/tfds/features)中找到有关特征的更多信息。在这里，我们特别可以检索图像的形状和类别数量：

In [ ]:
shape = features['image'].shape
num_classes = features['label'].num_classes

## 在纯 Python 中使用

您可以通过迭代来使用 Python 中的数据源：

In [ ]:
for example in ds['train']:
  print(example)
  break

如果您检查元素，还会注意到所有特征都已使用 NumPy 解码。在幕后，我们默认使用 [OpenCV](https://opencv.org)，因为它很快。如果您没有安装 OpenCV，我们将默认使用 [Pillow](python-pillow.org) 来提供轻量级和快速的图像解码。

```
{
  'image': array([[[0], [0], ..., [0]],
                  [[0], [0], ..., [0]]], dtype=uint8),
  'label': 2,
}
```

**注**：目前，该功能仅适用于 `Tensor`、`Image` 和 `Scalar` 特征。`Audio` 和 `Video` 特征即将推出。敬请关注！

## 与 PyTorch 结合使用

PyTorch 使用源/采样器/加载器范式。在 Torch 中，“数据源”称为“数据集”。[`torch.utils.data`](https://pytorch.org/docs/stable/data.html) 包含构建高效输入流水线所需的所有详细信息。

TFDS 数据源可以像常规的[映射样式数据集](https://pytorch.org/docs/stable/data.html#map-style-datasets)一样使用。

首先，我们安装并导入Torch：

In [ ]:
!pip install torch

from tqdm import tqdm
import torch

我们已经为训练和测试分别定义了数据源（分别是 `ds['train']` 和 `ds['test']`）。现在，我们可以定义采样器和加载器：

In [ ]:
batch_size = 128
train_sampler = torch.utils.data.RandomSampler(ds['train'], num_samples=5_000)
train_loader = torch.utils.data.DataLoader(
    ds['train'],
    sampler=train_sampler,
    batch_size=batch_size,
)
test_loader = torch.utils.data.DataLoader(
    ds['test'],
    sampler=None,
    batch_size=batch_size,
)

使用 PyTorch，我们在第一个样本上进行训练，并评估简单的逻辑回归：

In [ ]:
class LinearClassifier(torch.nn.Module):
  def __init__(self, shape, num_classes):
    super(LinearClassifier, self).__init__()
    height, width, channels = shape
    self.classifier = torch.nn.Linear(height * width * channels, num_classes)

  def forward(self, image):
    image = image.view(image.size()[0], -1).to(torch.float32)
    return self.classifier(image)


model = LinearClassifier(shape, num_classes)
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss()

print('Training...')
model.train()
for example in tqdm(train_loader):
  image, label = example['image'], example['label']
  prediction = model(image)
  loss = loss_function(prediction, label)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print('Testing...')
model.eval()
num_examples = 0
true_positives = 0
for example in tqdm(test_loader):
  image, label = example['image'], example['label']
  prediction = model(image)
  num_examples += image.shape[0]
  predicted_label = prediction.argmax(dim=1)
  true_positives += (predicted_label == label).sum().item()
print(f'\nAccuracy: {true_positives/num_examples * 100:.2f}%')

## 即将推出：与 JAX 结合使用

我们正在与 [Grain](https://github.com/google/grain) 密切合作。Grain 是适用于 Python 的开源、快速和确定性数据加载器。敬请关注！

## 阅读更多内容

有关详情，请参阅 [`tfds.data_source`](https://tensorflow.google.cn/datasets/api_docs/python/tfds/data_source) API 文档。